二：载入库与模型

In [35]:
#载入库与模型
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing#数据处理
#Within each family of models, you'll get an actual model after you fit and tune its parameters to the data.
from sklearn.ensemble import RandomForestRegressor#随机森林模型

from sklearn.pipeline import make_pipeline#交叉验证
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score#评价指标

from sklearn.externals import joblib#为模型以后使用


三：载入数据

In [36]:
#载入数据

dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'#远程载入红酒数据
data = pd.read_csv(dataset_url)
print (data.head())#前五行


  fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"
0   7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5                                                                                                                     
1   7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5                                                                                                                     
2  7.8;0.76;0.04;2.3;0.092;15;54;0.997;3.26;0.65;...                                                                                                                     
3  11.2;0.28;0.56;1.9;0.075;17;60;0.998;3.16;0.58...                                                                                                                     
4   7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5                                                                                                  

In [37]:
data=pd.read_csv(dataset_url,sep=';')#数据分隔开
print (data.head())


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [38]:
print(data.shape)#数据总量

(1599, 12)


In [39]:
print(data.describe())#数据总结
#count总数 mean平均数 std标准差 25%四分位点 50%中位数

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

四：分割数据为训练集与测试集

In [40]:
#把数据分为训练数据集与测试数据集
y=data.quality#把quality这一特性分割出来，作为输出
X=data.drop('quality', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.2,
                                                   random_state=123,#随机数种子，使用同样的种子抽取出来的样本一样
                                                   stratify=y)
#使用自带的数据集分割功能
#达成样本分层，保证测试集与训练集基本相同

五：数据处理

In [41]:
#不直接对训练集进行缩放的原因是，不能保证对测试集进行同样的缩放
scaler=preprocessing.StandardScaler().fit(X_train)#设置转换接口


In [42]:
X_train_scaled=scaler.transform(X_train)
print (X_train_scaled.mean(axis=0))

[  1.16664562e-16  -3.05550043e-17  -8.47206937e-17  -2.22218213e-17
   2.22218213e-17  -6.38877362e-17  -4.16659149e-18  -2.54439854e-15
  -8.70817622e-16  -4.08325966e-16  -1.17220107e-15]


In [43]:
print(X_train_scaled.std(axis=0))

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [44]:
#应用于测试集
X_test_scaled=scaler.transform(X_test)
print(X_test_scaled.mean(axis=0))

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]


In [45]:
print(X_test_scaled.std(axis=0))

[ 1.02160495  1.00135689  0.97456598  0.91099054  0.86716698  0.94193125
  1.03673213  1.03145119  0.95734849  0.83829505  1.0286218 ]


In [46]:
pipeline=make_pipeline(preprocessing.StandardScaler(),
                      RandomForestRegressor(n_estimators=100))

六：声明超参数调谐

In [47]:
#列出可调超参数
print (pipeline.get_params())

{'steps': [('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))], 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False), 'standardscaler__copy': True, 'standardscaler__with_mean': True,

In [48]:
#声明超参数调谐
hyperparameters={'randomforestregressor__max_features':['auto','sqrt','log2'],
                'randomforestregressor__max_depth':[None,5,3,1]}

七：使用交叉验证浅管道调参

In [49]:
#设置交叉验证
clf=GridSearchCV(pipeline, hyperparameters, cv=10)



In [50]:
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 'randomforestregressor__max_depth': [None, 5, 3, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [51]:
print(clf.best_params_)

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'sqrt'}


八：在整个训练集中重装

In [52]:
#确定模型重置
print(clf.refit)

True


九：在训练集中估算浅管道模型

In [54]:
#预测新的数据集
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))

0.46754646165


In [55]:
print(mean_squared_error(y_test,y_pred))

0.343578125


十：保存模型留着以后使用

In [57]:
#保存模型到pkl文件
joblib.dump(clf,'rf_regressor.pkl')

['rf_regressor.pkl']

In [58]:
#需要使用的时候直接从pkl文件中载入模型
clf2=joblib.load('rf_regressor.pkl')

In [59]:
clf2.predict(X_test)

array([ 6.48,  5.62,  4.98,  5.58,  6.35,  5.54,  5.1 ,  4.82,  5.02,
        6.02,  5.41,  5.76,  5.74,  5.08,  5.75,  5.62,  6.69,  5.7 ,
        5.7 ,  6.97,  5.44,  5.56,  5.01,  6.  ,  5.89,  5.1 ,  5.5 ,
        5.2 ,  5.95,  5.98,  5.95,  6.44,  6.  ,  5.05,  4.97,  5.93,
        5.08,  6.15,  5.06,  6.13,  4.97,  5.91,  6.47,  5.16,  6.16,
        5.32,  5.51,  5.56,  5.12,  6.42,  6.12,  5.26,  5.8 ,  5.15,
        5.61,  5.73,  5.3 ,  5.35,  5.01,  5.25,  5.41,  5.23,  5.13,
        5.75,  5.95,  5.19,  6.41,  5.07,  5.22,  6.72,  5.73,  5.64,
        5.1 ,  5.03,  5.28,  6.  ,  5.35,  5.12,  5.41,  5.36,  6.31,
        5.66,  6.25,  6.34,  5.1 ,  6.02,  6.39,  6.3 ,  5.7 ,  5.75,
        5.94,  5.37,  6.38,  5.62,  5.71,  5.81,  6.73,  6.73,  5.56,
        6.65,  5.06,  5.57,  5.19,  6.39,  5.03,  4.83,  5.55,  5.02,
        5.76,  5.99,  5.81,  5.44,  6.01,  5.37,  5.21,  5.25,  5.91,
        5.04,  5.03,  6.02,  5.86,  5.11,  5.71,  6.17,  5.36,  5.31,
        5.29,  6.01,